In [2]:
!pip install opencv-python

In [2]:
from ssd import build_ssd
from layers.box_utils import *
import os
import sys
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse
import torchvision
import pickle
from layers import box_utils
from layers import Detect
from layers import functions
from layers import modules
import torch.nn.functional as F
from math import sqrt as sqrt
from itertools import product as product

from torch.autograd import Function
from layers.box_utils import decode, nms

In [3]:
## 詳細模型結構可以參考ssd.py
ssd_net=build_ssd('train', size=300, num_classes=21)
ssd_net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')

Loading weights into state dict...
Finished!


C:\Users\SCU\Documents\Object Detection 程式導讀\Object Detection 程式導讀\ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)


## 默認Config檔案在data/config.py內

In [4]:
ssd_net.cfg

{'num_classes': 21,
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'min_dim': 300,
 'steps': [8, 16, 32, 64, 100, 300],
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'variance': [0.1, 0.2],
 'clip': True,
 'name': 'VOC'}

In [5]:
cfg = {
    'num_classes': 21,
    'lr_steps': (80000, 100000, 120000),
    'max_iter': 120000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC',
}

### 'aspect_ratios' : 使用六張Feature Map，每一張上方有預設的anchor boxes，Boxes aspect ratio可以自己設定
### 'feature_maps' : 使用feature map大小為[38x38, 19x19, 10x10, 5x5, 3x3, 1x1]
### 'min_sizes'、'max_sizes'可藉由下方算式算出，由作者自行設計
### 'steps' : Feature map回放回原本300*300的比例，如38要回放為300大概就是8倍
### 'variance' : Training 的一個trick，加速收斂，詳見：https://github.com/rykov8/ssd_keras/issues/53

---

## 'min_sizes'、'max_sizes' 計算

In [6]:
import math
## source:https://blog.csdn.net/gbyy42299/article/details/81235891
min_dim = 300   ## 维度
# conv4_3 ==> 38 x 38
# fc7 ==> 19 x 19
# conv6_2 ==> 10 x 10
# conv7_2 ==> 5 x 5
# conv8_2 ==> 3 x 3
# conv9_2 ==> 1 x 1
mbox_source_layers = ['conv4_3', 'fc7', 'conv6_2', 'conv7_2', 'conv8_2', 'conv9_2'] ## prior_box來源層，可以更改。很多改進都是基於此處的調整。
# in percent %
min_ratio = 20 ## 這裡即是論文中所說的Smin的= 0.2，Smax的= 0.9的初始值，經過下面的運算即可得到min_sizes，max_sizes。
max_ratio = 90
step = int(math.floor((max_ratio - min_ratio) / (len(mbox_source_layers) - 2)))## 取一個間距步長，即在下面用於循環給比取值時起一個間距作用。可以用一個具體的數值代替，這裡等於17。
min_sizes = []  ## 經過以下運算得到min_sizes和max_sizes。
max_sizes = []
for ratio in range(min_ratio, max_ratio + 1, step):
    ## 從min_ratio至max_ratio + 1每隔步驟= 17取一個值賦值給比。注意範圍函數的作用。
    ## min_sizes.append（）函數即把括號內部每次得到的值依次給了min_sizes。
    min_sizes.append(min_dim * ratio / 100.)
    max_sizes.append(min_dim * (ratio + step) / 100.)
min_sizes = [min_dim * 10 / 100.] + min_sizes
max_sizes = [min_dim * 20 / 100.] + max_sizes

## steps: 這一步要仔細理解，即計算卷積層產生的prior_box距離原圖的步長，先驗框中心點的坐標會乘以step，
## 相當於從特徵映射位置映射回原圖位置，比如conv4_3輸出特徵圖大小為38 *38，而輸入的圖片為300* 300，
## 所以38 *8約等於300，所以映射步長為8.這是針對300* 300的訓練圖片。
steps = [8, 16, 32, 64, 100, 300]  
aspect_ratios = [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
 
print('min_sizes: ',min_sizes)
print('max_sizes: ',max_sizes)


min_sizes:  [30.0, 60.0, 111.0, 162.0, 213.0, 264.0]
max_sizes:  [60.0, 111.0, 162.0, 213.0, 264.0, 315.0]


---

## Default anchor boxes設計原理，看懂收穫很多
##### 可以理解 SSD原文中 8732個anchors是怎麼來的
##### 38×38×4+19×19×6+10×10×6+5×5×6+3×3×4+1×1×4=8732

In [7]:
class PriorBox(object):
    """Compute priorbox coordinates in center-offset form for each source
    feature map.
    """
    def __init__(self, cfg):
        super(PriorBox, self).__init__()
        self.image_size = cfg['min_dim']
        # number of priors for feature map location (either 4 or 6)
        self.num_priors = len(cfg['aspect_ratios'])
        self.variance = cfg['variance'] or [0.1]
        self.feature_maps = cfg['feature_maps']
        self.min_sizes = cfg['min_sizes']
        self.max_sizes = cfg['max_sizes']
        self.steps = cfg['steps']
        self.aspect_ratios = cfg['aspect_ratios']
        self.clip = cfg['clip']
        self.version = cfg['name']
        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        mean = []
        '''依照Feature map大小找出所有的pixel 中心'''
        '''下方這兩個loop會找出W個x軸pixel對上W個y軸pixel，假如現在是在38x38的feature map上，就會有38x38個值'''
        '''ex. [0,1],[0,2]..[0,37] [1,1],[1,2]..[1,37]..........[37,37]'''
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                f_k = self.image_size / self.steps[k] ## 如self.steps==8，就是先將原圖size normalize(/300)後再乘上8
                # unit center x,y
                '''中心點'''
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # aspect_ratio: 1
                # rel size: min_size
                '''/self.image_size 就是在做normalization '''
                s_k = self.min_sizes[k]/self.image_size
                '''小的正方形box'''
                mean += [cx, cy, s_k, s_k]

                # aspect_ratio: 1
                # rel size: sqrt(s_k * s_(k+1))
                '''大的正方形box'''
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    '''aspect ratio 2,3'''
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    '''aspect ratio 1/2,1/3'''
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        # back to torch land
        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output

In [8]:
PriorBox_Demo=PriorBox(cfg)

In [9]:
print(PriorBox_Demo.forward().shape)

torch.Size([8732, 4])


---

## Loss 如何設計

In [10]:
class MultiBoxLoss(nn.Module):

    def __init__(self, num_classes, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        '''有幾類'''
        self.num_classes = num_classes
        '''判定為正樣本的threshold，一般設為0.5'''
        self.threshold = overlap_thresh
        '''background自己會有一類，不用Label，假如我們有20類一樣標註0-19，下方會自己空出一類給background'''
        self.background_label = bkg_label
        self.encode_target = encode_target
        self.use_prior_for_matching = prior_for_matching
        '''OHEM，找出分得最不好的樣品，也就是confidence score比較低的正負樣品'''
        self.do_neg_mining = neg_mining
        '''負樣品與正樣品的比例，通常是3:1'''
        self.negpos_ratio = neg_pos
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']
     

    def forward(self, predictions, targets):

        '''prediction會output三個值'''
        '''loc shape: bounding box 資訊，torch.size(batch_size,num_priors,4)'''
        '''conf shape: 每一個bounding box 的信心程度，torch.size(batch_size,num_priors,num_classes)'''
        '''priors shape: 預設的defaul box， torch.size(num_priors,4)'''
        loc_data, conf_data, priors = predictions
        num = loc_data.size(0)
        priors = priors[:loc_data.size(1), :]
        num_priors = (priors.size(0))
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            defaults = priors.data
            '''jaccard 計算每一個BBOX與ground truth的IOU'''
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        '''用Variable包裝'''
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        pos = conf_t > 0
        num_pos = pos.sum(dim=1, keepdim=True)


        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        '''smooth_l1_loss 計算bounding box regression'''
        loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        loss_c = loss_c.view(num, -1)
        loss_c[pos] = 0
        '''排列confidence 的分數'''
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        '''負樣品取出數量 == negpos_ratio*num_pos'''
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        '''用cross_entropy做分類'''
        loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        #double轉成torch.float64
        N = num_pos.data.sum().double()
        loss_l = loss_l.double()
        loss_c = loss_c.double()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c


## 產生我們Loss function，注意這裡的class要包含背景

In [11]:
'''要不要使用gpu'''
Use_cuda=True
criterion = MultiBoxLoss(21, 0.5, True, 0, False, 3, 0.5,False, Use_cuda,)

----

## 基本設定

In [39]:
ssd_net=build_ssd('train', size=300, num_classes=21)
use_pretrained=True
if use_pretrained:
    ssd_net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')
    
net=ssd_net

Loading weights into state dict...
Finished!


In [40]:
'''tensor type會依照cpu或gpu有所不同'''
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
#     if args.cuda:
#         torch.set_default_tensor_type('torch.cuda.FloatTensor')
#     if not args.cuda:
#         print("WARNING: It looks like you have a CUDA device, but aren't " +
#               "using CUDA.\nRun with --cuda for optimal training speed.")
#         torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

'''使用GPU時可以開啟DataParallel，但當Input是不定大小時，要關掉'''
if Use_cuda:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True
    
'''使用GPU時模型要轉成cuda'''
if Use_cuda:
    net = net.cuda()
    
batch_size_=1
optimizer = optim.Adam(net.parameters(),lr=0.00001/batch_size_)

---

## 訓練

## 這裡我們先示範輸入的 image,Label格式，真正在訓練時，準備成一樣格式即可

In [41]:
'''輸入影像格式，假設batch size 為 4'''
image_in = torch.tensor(torch.rand(4, 3, 300, 300), dtype=torch.float32)

'''Label格式，沒有固定長度，看圖像中有幾個label就有幾個'''
label_0=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
         [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
         [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
         [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_1=[[ 0.1804,  0.6076,  0.7701,  0.8485, 13.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 11.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 7.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 5.0000],]
label_2=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 14.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_3=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]

C:\Users\SCU\AppData\Local\Temp/ipykernel_14332/2364601204.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image_in = torch.tensor(torch.rand(4, 3, 300, 300), dtype=torch.float32)


In [42]:
epochs=300
iteration=1000

In [43]:
print("Use_cuda:", Use_cuda)

'''要用Variable包裝tensor才能送入模型'''
if Use_cuda:
    image_ = Variable(image_in.cuda())
    y = [Variable(torch.tensor(label_0).cuda(), volatile=True),
         Variable(torch.tensor(label_1).cuda(), volatile=True),
         Variable(torch.tensor(label_2).cuda(), volatile=True),
         Variable(torch.tensor(label_3).cuda(), volatile=True)]      
else:
    image_ = Variable(image_in)
    y = [Variable(torch.tensor(label_0), volatile=True),
         Variable(torch.tensor(label_1), volatile=True),
         Variable(torch.tensor(label_2), volatile=True),
         Variable(torch.tensor(label_3), volatile=True)]

print(image_.is_cuda)
for i in y:
    print(i.is_cuda)

Use_cuda: True
True
True
True
True
True


C:\Users\SCU\AppData\Local\Temp/ipykernel_14332/796295594.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  y = [Variable(torch.tensor(label_0).cuda(), volatile=True),
C:\Users\SCU\AppData\Local\Temp/ipykernel_14332/796295594.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  Variable(torch.tensor(label_1).cuda(), volatile=True),
C:\Users\SCU\AppData\Local\Temp/ipykernel_14332/796295594.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  Variable(torch.tensor(label_2).cuda(), volatile=True),
C:\Users\SCU\AppData\Local\Temp/ipykernel_14332/796295594.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  Variable(torch.tensor(label_3).cuda(), volatile=True)]


In [44]:
out = net(image_)

In [45]:
for o in out:
    print(o.is_cuda, o.shape)

True torch.Size([4, 8732, 4])
True torch.Size([4, 8732, 21])
True torch.Size([8732, 4])


In [46]:
n=0
loss_sum=[]
loc_loss=[]
conf_loss=[]
    
'''Regression Loss and Classification Loss'''
loss_l, loss_c = criterion(out,y )
loss = loss_l + loss_c

'''Backward'''
loss.backward()

loc_loss.append(loss_l.data.cpu().numpy())
conf_loss.append(loss_c.data.cpu().numpy())
loss_sum.append(loss.data.cpu().numpy())

'''更新參數'''
optimizer.step()

'''清空Gradients'''
optimizer.zero_grad()

n += 1
if n % 10 == 0:
    print('BBOX Regression Loss: ', np.mean(loc_loss))
    print('Classification Loss: ', np.mean(conf_loss))

C:\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [47]:
for epoch in range(epochs):
    n=0
    loss_sum=[]
    loc_loss=[]
    conf_loss=[]
    for number__ in range(iteration) :
        '''要用Variable包裝tensor才能送入模型'''
        if Use_cuda:
            image_ = Variable(image_in.cuda())
            y = [Variable(torch.tensor(label_0).cuda(), volatile=True),Variable(torch.tensor(label_1).cuda(), 
                volatile=True),Variable(torch.tensor(label_2).cuda(), volatile=True),Variable(torch.tensor(label_3).cuda(), volatile=True)]      
        else:
            image_ = Variable(image_in)
            y = [Variable(torch.tensor(label_0), volatile=True),Variable(torch.tensor(label_1), 
                volatile=True),Variable(torch.tensor(label_2), volatile=True),Variable(torch.tensor(label_3), volatile=True)]

        '''Forward Pass'''
        out = net(image_)
        '''Regression Loss and Classification Loss'''
        loss_l,loss_c = criterion(out,y )
        loss = loss_l+ loss_c
        '''Backward'''
        loss.backward()

        loc_loss.append(loss_l.data.cpu().numpy())
        conf_loss.append(loss_c.data.cpu().numpy())
        loss_sum.append(loss.data.cpu().numpy())
        '''更新參數'''
        optimizer.step()
        '''清空Gradients'''
        optimizer.zero_grad()
        
        n+=1
        if n%10==0:
            print('BBOX Regression Loss: ', np.mean(loc_loss))
            print('Classification Loss: ', np.mean(conf_loss))
    '''儲存權重'''
#     torch.save(ssd_net.state_dict(),'weights/Ｗeights.pth')

C:\Users\SCU\AppData\Local\Temp/ipykernel_14332/1317199139.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  y = [Variable(torch.tensor(label_0).cuda(), volatile=True),Variable(torch.tensor(label_1).cuda(),
C:\Users\SCU\AppData\Local\Temp/ipykernel_14332/1317199139.py:11: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  volatile=True),Variable(torch.tensor(label_2).cuda(), volatile=True),Variable(torch.tensor(label_3).cuda(), volatile=True)]


BBOX Regression Loss:  1.886542468600803
Classification Loss:  7.459711654097946
BBOX Regression Loss:  1.6990058263142906
Classification Loss:  5.9205705289487485
BBOX Regression Loss:  1.5861289930932319
Classification Loss:  5.17972499706127
BBOX Regression Loss:  1.452247225796735
Classification Loss:  4.6805721777456775
BBOX Regression Loss:  1.321750147784198
Classification Loss:  4.281830026132089
BBOX Regression Loss:  1.2000414053599038
Classification Loss:  3.9341670118732215
BBOX Regression Loss:  1.0921298602270697
Classification Loss:  3.614457024468316
BBOX Regression Loss:  0.993659069803026
Classification Loss:  3.3142375027691875
BBOX Regression Loss:  0.9031764990998883
Classification Loss:  3.0391331786481435
BBOX Regression Loss:  0.8242397668626574
Classification Loss:  2.7895513414453577
BBOX Regression Loss:  0.7565716137789719
Classification Loss:  2.567232610721781
BBOX Regression Loss:  0.6980102957398804
Classification Loss:  2.371226466143573
BBOX Regression

BBOX Regression Loss:  0.0860276304011546
Classification Loss:  0.2916058880978345
BBOX Regression Loss:  0.08517339336513531
Classification Loss:  0.28869221955694746
BBOX Regression Loss:  0.00021735312098085328
Classification Loss:  0.0002364315678951917
BBOX Regression Loss:  0.0001489925340542363
Classification Loss:  0.00023276224146010703
BBOX Regression Loss:  0.00010998371397834961
Classification Loss:  0.00022816152806267326
BBOX Regression Loss:  8.574910176253912e-05
Classification Loss:  0.0002233870741393831
BBOX Regression Loss:  7.008158966696269e-05
Classification Loss:  0.00021865971800353795
BBOX Regression Loss:  5.973196585492351e-05
Classification Loss:  0.00021412961736873344
BBOX Regression Loss:  5.1967728647887563e-05
Classification Loss:  0.0002097750459575937
BBOX Regression Loss:  4.615914412742886e-05
Classification Loss:  0.0002056337824022329
BBOX Regression Loss:  4.215337723635382e-05
Classification Loss:  0.00020169136938428191
BBOX Regression Loss:  

BBOX Regression Loss:  0.00038693407651864463
Classification Loss:  0.00014081728016585417
BBOX Regression Loss:  0.0003839630726623222
Classification Loss:  0.0001404931493764878
BBOX Regression Loss:  0.00038028295798052896
Classification Loss:  0.0001401361649019969
BBOX Regression Loss:  0.0003764161334458064
Classification Loss:  0.00013973768230805055
BBOX Regression Loss:  0.00037253969684668134
Classification Loss:  0.00013929923087699779
BBOX Regression Loss:  0.00036870753839619077
Classification Loss:  0.00013882899534727597
BBOX Regression Loss:  0.0003649417765489352
Classification Loss:  0.00013832905178277284
BBOX Regression Loss:  0.00036125230100799235
Classification Loss:  0.00013780372953098703
BBOX Regression Loss:  0.0003576403150038395
Classification Loss:  0.00013725639361326518
BBOX Regression Loss:  0.0003540917501861124
Classification Loss:  0.00013669015881088043
BBOX Regression Loss:  2.295851461692817e-06
Classification Loss:  7.783606027563413e-05
BBOX Reg

BBOX Regression Loss:  0.00035042941308331276
Classification Loss:  7.831123506875575e-05
BBOX Regression Loss:  0.00034660418612240853
Classification Loss:  7.828797621905436e-05
BBOX Regression Loss:  0.00034269623126180146
Classification Loss:  7.821761269004678e-05
BBOX Regression Loss:  0.0003387799707220387
Classification Loss:  7.810835152704824e-05
BBOX Regression Loss:  0.00033491490340454194
Classification Loss:  7.796488158034518e-05
BBOX Regression Loss:  0.00033113423221669663
Classification Loss:  7.779211890084124e-05
BBOX Regression Loss:  0.00032744811558929566
Classification Loss:  7.759455162791525e-05
BBOX Regression Loss:  0.00032385004836869635
Classification Loss:  7.737561968852739e-05
BBOX Regression Loss:  0.0003203317475942844
Classification Loss:  7.713809272059834e-05
BBOX Regression Loss:  0.0003169058659374929
Classification Loss:  7.688442811080452e-05
BBOX Regression Loss:  0.00031367730262729703
Classification Loss:  7.661665833456132e-05
BBOX Regressi

...

BBOX Regression Loss:  3.2534746394587813e-06
Classification Loss:  1.333745827087128e-10
BBOX Regression Loss:  3.1984106960104926e-06
Classification Loss:  1.280395994003643e-10
BBOX Regression Loss:  3.367255369131528e-06
Classification Loss:  1.3160568905628717e-10
BBOX Regression Loss:  3.6765492663561706e-06
Classification Loss:  1.389957337461113e-10
BBOX Regression Loss:  3.7103374087944746e-06
Classification Loss:  1.576842353339244e-10
BBOX Regression Loss:  3.6420137517619655e-06
Classification Loss:  1.5985919033807731e-10
BBOX Regression Loss:  3.565086315036212e-06
Classification Loss:  1.545305506601414e-10
BBOX Regression Loss:  3.5244259757537848e-06
Classification Loss:  1.495456941872336e-10
BBOX Regression Loss:  3.5536799353403024e-06
Classification Loss:  1.4487239124388255e-10
BBOX Regression Loss:  3.607750556290242e-06
Classification Loss:  1.4048231878194673e-10
BBOX Regression Loss:  3.533056113251018e-06
Classification Loss:  1.3635048587659534e-10
BBOX Regr

BBOX Regression Loss:  3.288232427941559e-06
Classification Loss:  9.65815945737821e-11
BBOX Regression Loss:  3.2152738363028315e-06
Classification Loss:  9.09003243047361e-11
BBOX Regression Loss:  3.406094748680954e-06
Classification Loss:  8.585030628780632e-11
BBOX Regression Loss:  3.376506871492652e-06
Classification Loss:  8.714128833724699e-11
BBOX Regression Loss:  3.2717023112935642e-06
Classification Loss:  8.278422392038465e-11
BBOX Regression Loss:  3.1818778392001295e-06
Classification Loss:  7.884211801941395e-11
BBOX Regression Loss:  3.193061555292987e-06
Classification Loss:  7.525838538216786e-11
BBOX Regression Loss:  3.5866209000759874e-06
Classification Loss:  8.638353800387964e-11
BBOX Regression Loss:  3.7329765542594546e-06
Classification Loss:  8.738334747151713e-11
BBOX Regression Loss:  3.685743420646759e-06
Classification Loss:  8.388801357265644e-11
BBOX Regression Loss:  3.6091880784777544e-06
Classification Loss:  8.066155151216967e-11
BBOX Regression L

BBOX Regression Loss:  3.2245340325634633e-06
Classification Loss:  2.7594741306794884e-11
BBOX Regression Loss:  3.5365392235270603e-06
Classification Loss:  3.679298840905984e-11
BBOX Regression Loss:  3.474323128574286e-06
Classification Loss:  4.4151586090871815e-11
BBOX Regression Loss:  3.4669332975687887e-06
Classification Loss:  4.0137805537156197e-11
BBOX Regression Loss:  3.646923083399514e-06
Classification Loss:  3.679298840905985e-11
BBOX Regression Loss:  3.6664267358810577e-06
Classification Loss:  3.396275853143986e-11
BBOX Regression Loss:  3.598556237607251e-06
Classification Loss:  3.153684720776558e-11
BBOX Regression Loss:  3.566835081904702e-06
Classification Loss:  2.943439072724788e-11
BBOX Regression Loss:  3.5048541020741038e-06
Classification Loss:  3.4493426633493605e-11
BBOX Regression Loss:  3.453198849075942e-06
Classification Loss:  3.895728184488689e-11
BBOX Regression Loss:  3.5319597208335874e-06
Classification Loss:  4.905731787874646e-11
BBOX Regres

BBOX Regression Loss:  3.7365878823847095e-06
Classification Loss:  1.0817138572526297e-10
BBOX Regression Loss:  3.559244403193049e-06
Classification Loss:  3.311368956815386e-10
BBOX Regression Loss:  2.7872583140614787e-06
Classification Loss:  4.4151585761916844e-10
BBOX Regression Loss:  2.263712874228526e-06
Classification Loss:  2.943439050794456e-10
BBOX Regression Loss:  2.133587843302478e-06
Classification Loss:  2.2075792880958422e-10
BBOX Regression Loss:  3.2420180394654856e-06
Classification Loss:  1.9868213609310326e-10
BBOX Regression Loss:  3.290078061750305e-06
Classification Loss:  1.6556844674425273e-10
BBOX Regression Loss:  3.060260164530187e-06
Classification Loss:  1.4191581149507375e-10
BBOX Regression Loss:  2.825967997658288e-06
Classification Loss:  1.2417633505818953e-10
BBOX Regression Loss:  2.6997922100890866e-06
Classification Loss:  1.226432939658551e-10
BBOX Regression Loss:  2.5471323257348824e-06
Classification Loss:  1.2141686109198755e-10
BBOX Reg

BBOX Regression Loss:  3.472049715364173e-06
Classification Loss:  1.1157873651683259e-10
BBOX Regression Loss:  3.460592513987879e-06
Classification Loss:  1.1037896515643654e-10
BBOX Regression Loss:  3.494742726855474e-06
Classification Loss:  1.1155320946811658e-10
BBOX Regression Loss:  3.552043188633531e-06
Classification Loss:  1.1386461669141575e-10
BBOX Regression Loss:  3.553270723918901e-06
Classification Loss:  1.1497808870977973e-10
BBOX Regression Loss:  3.5428792544228527e-06
Classification Loss:  1.1379274758906035e-10
BBOX Regression Loss:  3.5245624998065466e-06
Classification Loss:  1.1263159710345769e-10
BBOX Regression Loss:  3.5498090791952287e-06
Classification Loss:  1.1149390420342276e-10
BBOX Regression Loss:  3.5392542317317245e-06
Classification Loss:  1.1037896516138854e-10
